### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# This was all pre-set from the downloaded file
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [10]:
#Show me the schools 
unique_schools = school_data_complete["school_name"].unique()

#Count all schools
nr_schools = len(unique_schools)

#Count all students
total_students = school_data_complete["student_name"].count()

# Show me the total of School budgets, we can sum unique budgets assuming all schools have different budgets
# This was confirmed later to be true in the summary table
total_budget = sum(school_data_complete["budget"].unique())

#Calculate average math & reading scores
average_math = school_data_complete["math_score"].mean()
average_reading = school_data_complete["reading_score"].mean()

#Show me students with at least 70 score in maths. Then count them and divide by total students to get %
math_passing = school_data_complete[school_data_complete["math_score"] >= 70] 
math_passing = math_passing["math_score"].count()/total_students*100 #this could be done in 1 step only

#Show me students with at least 70 score in reading. Then count them and divide by total students to get %
reading_passing = school_data_complete[school_data_complete["reading_score"] >= 70]
reading_passing = reading_passing["reading_score"].count()/total_students*100

#Show me students with at least 70 score in maths AND reading. Then count them and divide by total students to get %
overall_passing = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)]
# We just need to count either math or reading as overall dataframe only shows students that passed both
overall_passing = overall_passing["math_score"].count()/total_students*100

In [11]:
#Do a new summary table
summary_df = pd.DataFrame({
    "Total Schools": [nr_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math],
    "Average Reading Score": [average_reading],
    "% Passing Math":[math_passing],
    "% Passing Reading": [reading_passing],
    "% Overall Passing": [overall_passing]})

summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [12]:
#format cells 
summary_format_df = summary_df.style.format({"Total Students": "{:,}",
                                             "Average Reading Score": "{:,.5f}",
                                            "Total Budget": "${:,.2f}"}) 
summary_format_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [27]:
# Group Data Frame by Schools
grouped_school = school_data_complete.set_index("school_name").groupby(["school_name"])

# We have now to produce other columns where the values need to fit appropriately into this grouped table

# Write School type, we just need to grab the first value for each school
school_type = grouped_school["type"].first()

# count students from grouped data
Student_per_school = grouped_school["student_name"].count()

# Write School budget, just grab the first value of each, we will see now if all budgets are different...
school_budget = grouped_school["budget"].first()

# Divide budget by number of students, both are listed already
budget_per_student = school_budget/Student_per_school

# Calculate averages per grouped data
average_math_per_school = grouped_school["math_score"].mean()
average_reading_per_school = grouped_school["reading_score"].mean()

# See how many passed per school, syntax is challenging here! Easier if we let this happen in 2 steps
# It works because data is clean
percent_passing_math = school_data_complete[school_data_complete["math_score"]>=70].groupby(["school_name"]).size()/Student_per_school*100
percent_passing_reading = school_data_complete[school_data_complete["reading_score"]>=70].groupby(["school_name"]).size()/Student_per_school*100
# Doing for both with some creativity, it seems to work here! Tried this above but syntax wasn't working
percent_passing = school_data_complete[(school_data_complete["reading_score"] >= 70) & 
(school_data_complete["math_score"] >= 70)].groupby("school_name")["student_name"].count()/Student_per_school*100


In [28]:
#Do Summary table

summary_table = pd.DataFrame({
    "School Type": school_type,
    "Total Students": Student_per_school,
    "Total School Budget": school_budget,
    "Per Student Budget": budget_per_student,
    "Average Math Score": average_math_per_school,
    "Average Reading Score": average_reading_per_school,
    "% Passing Math": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "% Overall Passing": percent_passing})

#And format
summary_table_format = summary_table.style.format({"Total School Budget": "${:,.2f}",
                                             "Per Student Budget": "${:,.2f}"}) 
summary_table_format



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [13]:
#do sort table, do not use format table directly, use numbers first and then format again!
top_five = summary_table.sort_values(by=["% Overall Passing"], ascending=False).head(5)
top_five

#there we go, re-format
top_five_format = top_five.style.format({"Total School Budget": "${:,.2f}",
                                             "Per Student Budget": "${:,.2f}"}) 
top_five_format


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,['Charter'],1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,['Charter'],1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,['Charter'],1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,['Charter'],2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,['Charter'],962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [14]:
#for bottom same thing but without telling the order, pandas already defaults to ascending order
bottom_five = summary_table.sort_values(by=["% Overall Passing"]).head(5)

#and re-format
bottom_five_format = bottom_five.style.format({"Total School Budget": "${:,.2f}",
                                             "Per Student Budget": "${:,.2f}"}) 
bottom_five_format

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,['District'],3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,['District'],2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,['District'],2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,['District'],4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,['District'],4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:
#Average math for each grade and by school, this will give us columns of average for each school
avg_math_9th = school_data_complete[school_data_complete["grade"]=="9th"].groupby("school_name")["math_score"].mean()
avg_math_10th = school_data_complete[school_data_complete["grade"]=="10th"].groupby("school_name")["math_score"].mean()
avg_math_11th = school_data_complete[school_data_complete["grade"]=="11th"].groupby("school_name")["math_score"].mean()
avg_math_12th = school_data_complete[school_data_complete["grade"]=="12th"].groupby("school_name")["math_score"].mean()

In [16]:
#creating new dataframe with all grade columns
grade_math=pd.DataFrame({"9th":avg_math_9th,"10th":avg_math_10th,"11th":avg_math_11th,"12th":avg_math_12th})

In [17]:
#optional and cleaner formatting
grade_math_format = grade_math.style.format({"9th": "{:,.2f}",
                                             "10th": "{:,.2f}",
                                             "11th": "{:,.2f}",
                                             "12th": "{:,.2f}"}) 
grade_math_format

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [18]:
#Same as Maths, just replacing word maths for reading
avg_reading_9th = school_data_complete[school_data_complete["grade"]=="9th"].groupby("school_name")["reading_score"].mean()
avg_reading_10th = school_data_complete[school_data_complete["grade"]=="10th"].groupby("school_name")["reading_score"].mean()
avg_reading_11th = school_data_complete[school_data_complete["grade"]=="11th"].groupby("school_name")["reading_score"].mean()
avg_reading_12th = school_data_complete[school_data_complete["grade"]=="12th"].groupby("school_name")["reading_score"].mean()


grade_reading=pd.DataFrame({"9th":avg_reading_9th,"10th":avg_reading_10th,"11th":avg_reading_11th,"12th":avg_reading_12th})

In [19]:
#optional and cleaner formatting

grade_reading_format = grade_reading.style.format({"9th": "{:,.2f}",
                                             "10th": "{:,.2f}",
                                             "11th": "{:,.2f}",
                                             "12th": "{:,.2f}"}) 
grade_reading_format

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [20]:
# Define spending values
spending = [0, 584, 630, 645, 675]
# Write labels
spending_class = ["<$584", "$585-629", "$630-644", "$645-675"]

In [21]:
# Filtering the columns we need, work with summary table this is now to rate school and we already calculated budget per student
spending_ranges = summary_table.loc[:,["Average Math Score",
                                  "Average Reading Score","% Passing Math",
                                  "% Passing Reading","% Overall Passing",]]
# Add a new column for spending and alocate  values
spending_ranges["Spending Ranges (Per Student)"]= pd.cut(summary_table["Per Student Budget"],spending,labels=spending_class)
# Group by the spending, in a way it is the inverted process we did for the summary table
spending_ranges = spending_ranges.groupby("Spending Ranges (Per Student)").mean()
spending_ranges

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


In [22]:
# format to a more readableway

spending_ranges_format = spending_ranges.style.format({"Average Math Score": "{:,.2f}",
                                             "Average Reading Score": "{:,.2f}",
                                             "% Passing Math": "{:,.2f}",
                                             "% Passing Reading": "{:,.2f}",
                                             "% Overall Passing": "{:,.2f}"}) 
spending_ranges_format

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [23]:
# Define spending values in similar fashion
sizes = [0,1000, 2000, 5000]
# Write labels
size_class = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

In [24]:
# Columns for the new data frame
size_ranges = summary_table.loc[:,["Average Math Score",
                                  "Average Reading Score","% Passing Math",
                                  "% Passing Reading","% Overall Passing",]]

# Add a new column for spending and alocate values, note that size refers to total nr of students 
size_ranges["School Size"]= pd.cut(summary_table["Total Students"],sizes,labels=size_class)
# Group by the spending, in a way it is the inverted process we did for the summary table
size_ranges = size_ranges.groupby("School Size").mean()
size_ranges

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [30]:
# Just select columns for the new data frame 
# This is more simple, School Type classes are already defined, no need to tell what they are

type_ranges = summary_table.loc[:,["School Type","Average Math Score",
                                  "Average Reading Score","% Passing Math",
                                  "% Passing Reading","% Overall Passing",]]

type_ranges = type_ranges.groupby("School Type").mean()
type_ranges

#This last one was very interesting in my case, I had an error because way back I had selected unique types for summary table

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
